## 1. DB 저장

In [1]:
import os  # os를 가져와 파일 시스템 접근, 환경 변수 읽을 수 있음
from openai import OpenAI  # OpenAI의 api 사용 가능
import chromadb  # chromadb 라이브러리 쓸 수 있게 해줌
from chromadb.config import (
    Settings,
)  # Settings 클래스는 DB의 구성 옵션을 설정하는데 사용
from dotenv import load_dotenv  # 환경 변수를 로드하기 위함

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [3]:
def init_db(db_path="./chroma_db"): # 현재 디렉토리 내의 chroma_db폴더 설정
    dbclient = chromadb.PersistentClient(path=db_path) # 지정 경로로 향하는 dbClient생성
    collection = dbclient.create_collection(name="rag_collection", get_or_create=True)
    return dbclient, collection

In [4]:
import pandas as pd

def load_csv_files(folder_path):
    docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, filename))
            # 멜론 차트 >> Rank, Title, Artist, Album
            # 'Title' + 'Artist'를 합쳐서 하나의 문장으로 만들기
            for idx, row in df.iterrows():
                text = f"{row['Rank']}위: {row['Title']} - {row['Artist']} (앨범 {row['Album']})"
                docs.append((filename, text))
    return docs            

In [5]:
def get_embedding(text, model="text-embedding-3-large"):
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding


In [6]:
def chunk_text(text, chunk_size=400, chunk_overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start+chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - chunk_overlap
        
        if start<0:
            start = 0

        if start >= len(text):
            break

    return chunks

In [7]:
if __name__ == "__main__":
    # db 초기화, 경로 지정
    dbclient, collection = init_db("./chroma_db")

    # CSV 파일 불러오기
    folder_path = "./crawling_source"
    docs = load_csv_files(folder_path)

    doc_id = 0

    for filename, text in docs:
        doc_id += 1
        embedding = get_embedding(text)

        collection.add(
            documents=[text],
            embeddings=[embedding],
            metadatas=[{"filename": filename}],
            ids=[str(doc_id)]
        )

    print("모든 문서 벡터DB에 저장 완료")

모든 문서 벡터DB에 저장 완료
